In [3]:
import os
os.environ['TORCH_ROCM_AOTRITON_ENABLE_EXPERIMENTAL'] = '1'  # Для AMD GPU
import torch
print(torch.__version__)  # Должно вывести: 2.5.1+rocm6.2
print(torch.cuda.is_available())  # Проверка работы ROCm (True)


2.5.1+rocm6.2
True


In [4]:
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
import torch

print("Версия PyTorch:", torch.__version__)
print("Доступно GPU:", torch.cuda.is_available())

Версия PyTorch: 2.5.1+rocm6.2
Доступно GPU: True


In [5]:
# Инициализация модели для русских текстов
model = SentenceTransformer("sberbank-ai/sbert_large_mt_nlu_ru")
print("Модель загружена:", model.device)  # Проверка использования GPU

Using the `SDPA` attention implementation on multi-gpu setup with ROCM may lead to performance issues due to the FA backend. Disabling it to use alternative backends.


Модель загружена: cuda:0


In [7]:
# Пример использования BM25
# Подготовка корпуса документов
corpus = ["привет мир", "семантический поиск", "машинное обучение", "большие языковые модели"]
from rank_bm25 import BM25Okapi

# Токенизация корпуса
tokenized_corpus = [doc.split() for doc in corpus]

# Инициализация модели BM25
bm25 = BM25Okapi(tokenized_corpus)

# Пример запроса
query = "привет обучение"
tokenized_query = query.split()

# Расчет релевантности
scores = bm25.get_scores(tokenized_query)
top_docs = bm25.get_top_n(tokenized_query, corpus, n=2)

# Вывод результатов
print("\n▌ Результаты BM25:")
print(f"Запрос: '{query}'")
print("\nОценки релевантности:")
for doc, score in zip(corpus, scores):
    print(f"• '{doc}': {score:.2f}")

print("\nТоп-2 документа:")
for i, doc in enumerate(top_docs, 1):
    print(f"{i}. {doc}")



▌ Результаты BM25:
Запрос: 'привет обучение'

Оценки релевантности:
• 'привет мир': 0.89
• 'семантический поиск': 0.00
• 'машинное обучение': 0.89
• 'большие языковые модели': 0.00

Топ-2 документа:
1. машинное обучение
2. привет мир


In [8]:
from natasha import Doc, Segmenter, NewsEmbedding, NewsMorphTagger, MorphVocab
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Инициализация моделей
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
morph_vocab = MorphVocab()
# sbert_model = SentenceTransformer('ai-forever/sbert_large_mt_nlu_ru')
sbert_model = model # Используйте предварительно загруженную модель

# Пример текста
text = "Машинное обучение позволяет компьютерам обучаться на данных. Искусственный интеллект меняет мир."

# Обработка текста с Natasha
doc = Doc(text)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)

# Лемматизация
for token in doc.tokens:
    token.lemmatize(morph_vocab)
lemmas = [token.lemma for token in doc.tokens]
print("Леммы:", lemmas)

# Извлечение триплетов (субъект-действие-объект)
triplets = []
for sent in doc.sents:
    subj = [token.lemma for token in sent.tokens if 'Subj' in sent.morph]
    obj = [token.lemma for token in sent.tokens if 'Obj' in sent.morph]
    if subj and obj:
        triplets.append((' '.join(subj), 'действие', ' '.join(obj)))
print("\nТриплеты:", triplets)

# Семантическое сравнение с SBERT
sentences = [
    "нейронные сети анализируют информацию",
    "искусственный интеллект обрабатывает данные",
    "погода в Москве сегодня солнечная"
]

embeddings = sbert_model.encode(sentences)
similarity = cosine_similarity([embeddings[0]], embeddings[1:])

print("\nСходство между предложениями:")
print(f"1 vs 2: {similarity[0][0]:.2f}")
print(f"1 vs 3: {similarity[0][1]:.2f}")


Леммы: ['машинный', 'обучение', 'позволять', 'компьютер', 'обучаться', 'на', 'данные', '.', 'искусственный', 'интеллект', 'менять', 'мир', '.']

Триплеты: []

Сходство между предложениями:
1 vs 2: 0.58
1 vs 3: -0.02


/home/gna/anaconda3/envs/rocm/lib/python3.10/site-packages/torch/nn/modules/linear.py:125: UserWarning: Attempting to use hipBLASLt on an unsupported architecture! Overriding blas backend to hipblas (Triggered internally at ../aten/src/ATen/Context.cpp:296.)
  return F.linear(input, self.weight, self.bias)


In [9]:
from natasha import Doc, Segmenter, NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, MorphVocab
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import re

# Инициализация моделей
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)  # Добавляем синтаксический парсер
morph_vocab = MorphVocab()
# sbert_model = SentenceTransformer('ai-forever/sbert_large_mt_nlu_ru')
sbert_model = model # Используйте предварительно загруженную модель

# Пример текста с исправлением
text = "Машинное обучение позволяет компьютерам обучаться на данных. Искусственный интеллект меняет мир."
print("Исходный текст:\n ", text)

# Обработка текста с Natasha
doc = Doc(text)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)  # Добавляем синтаксический анализ

# Лемматизация с фильтрацией пунктуации
lemmas = []
for token in doc.tokens:
    if token.pos != 'PUNCT':  # Игнорируем пунктуацию
        token.lemmatize(morph_vocab)
        lemmas.append(token.lemma)
print("Леммы:", lemmas)

# Извлечение триплетов (исправленная версия)
triplets = []
for sent in doc.sents:
    subj = [token.lemma for token in sent.tokens if token.rel == 'nsubj']
    obj = [token.lemma for token in sent.tokens if token.rel == 'obj']
    verb = [token.lemma for token in sent.tokens if token.rel == 'root']
    
    if subj and verb and obj:
        triplets.append((' '.join(subj), ' '.join(verb), ' '.join(obj)))

print("\nТриплеты:", triplets)

# Семантическое сравнение с SBERT
sentences = [
    "нейронные сети анализируют информацию",
    "искусственный интеллект обрабатывает данные",
    "погода в Москве сегодня солнечная"
]

embeddings = sbert_model.encode(sentences)
similarity = cosine_similarity([embeddings[0]], embeddings[1:])

print("\nСходство между предложениями:")
print(f"1 vs 2: {similarity[0][0]:.2f}")
print(f"1 vs 3: {similarity[0][1]:.2f}")


Исходный текст:
  Машинное обучение позволяет компьютерам обучаться на данных. Искусственный интеллект меняет мир.
Леммы: ['машинный', 'обучение', 'позволять', 'компьютер', 'обучаться', 'на', 'данные', 'искусственный', 'интеллект', 'менять', 'мир']

Триплеты: [('интеллект', 'менять', 'мир')]

Сходство между предложениями:
1 vs 2: 0.58
1 vs 3: -0.02


In [11]:
# Пример документов для поиска
documents = [
    "Твой лучший секс спрятан здесь 🔞  Делюсь каналом дипломированного сексолога. Крис взломала код классного секса, мастерски раскрепощает, знает миллион горячих техник и лучшие девайсы для взрослых 😻  Самые полезные посты здесь:   Отрезвляющий пост «Я все сама!»   Прокачай наездницу  Ролевая игра «VIP кинотеатр»   Техника оральных ласк 💣   Как занимается сeксом неудобная женщина   Кстати, Крис провела трехдневный безоплатный онлайн интенсив-«От бревна до Богини». Совместно с врачом и владельцем секс-шопа.   Скорее смотри записи, пока не удалила 🔞  https://t.me/sekretskris/1048   Здесь жарче, чем в аду 😈",
    "⭐️  Кнопка: ⭐️START⭐️(https://t.me/major/start?startapp=1972869792)",
    "Таро-прогноз на 23 Июля – Туз Мечей.   🔮 Совет: Меч – это достаточное количество сил для энергичных начинаний, огромная решимость действовать, начало успешной борьбы, готовность к росту, отрыву от изначального существования. Это ситуация, когда сам человек для себя многое прояснил и знает теперь, что хочет делать дальше. Это показатель силы – физической силы, силы воли, силы, приобретенной положением и умом или в силу обстоятельств. Триумф личной силы, власти над обстоятельствами. Триумф может относиться к любой стороне жизни: к работе, любви, денежным делам, духу, любым увлекающим занятиям. Этот Туз не столько начало, но и показатель завоеванного, торжества провозглашенных взглядов и принятых решений.   💰 Деньги: Процветание. Принятие однозначных и окончательных решений в этих вопросах. Возможность улучшить качество жизни, приняв вызов.   🩷 Любовь: В области отношений Туз Мечей символизирует импульсивную первобытную силу, интенсивные «завоевательные» эмоции, крайние чувства, связанные с ситуацией или человеком. Эмоции, которым покровительствует Туз Мечей, способны воспламенить таким огнем, который сожжет все препятствия на пути к цели, попутно причинив немало вреда. Мечи вообще масть холодная, и когда на горизонте в кои-то веки появляется единственная эмоция, то она заполоняет собой всё со свойственной этой масти тотальностью.  🎁🔥 РАСПРОДАЖА КУРСА «ОРАКУЛ ЛЕНОРМАН. БАЗОВЫЙ КУРС» В РАССРОЧКУ 👉🏻 http://alexeygrishin.com/lenorman_base.",
    "он вообще не собирается переезжать в другое государство",
    "ты не мог бы набрать меня после обеда"
]

In [12]:
# Установка зависимостей (если не установлены)
# !pip install natasha sentence-transformers rank_bm25

from natasha import Doc, Segmenter, NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, MorphVocab
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
from sklearn.metrics.pairwise import cosine_similarity
import re
import numpy as np

# Инициализация компонентов
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
morph_vocab = MorphVocab()
# sbert_model = SentenceTransformer('ai-forever/sbert_large_mt_nlu_ru')
sbert_model = model # Используйте предварительно загруженную модель


# def preprocess_text(text):
#     """Функция предобработки текста"""
#     # Приведение к нижнему регистру и замена ё
#     text = text.lower().replace("ё", "е")
#     # Удаление пунктуации и цифр
#     text = re.sub(r'[^\w\s]|[\d]', ' ', text)
#     # Удаление лишних пробелов
#     return re.sub(r'\s+', ' ', text).strip()

def preprocess_text(text):
    """Функция предобработки текста с расширенной очисткой"""
    # Приведение к нижнему регистру и замена ё
    text = text.lower().replace("ё", "е")
    
    # Удаление URL-ссылок
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
    
    # Удаление эмодзи и специальных символов
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # эмоции
        u'\U0001F300-\U0001F5FF'  # символы
        u'\U0001F680-\U0001F6FF'  # транспорт
        u'\U0001F700-\U0001F77F'  # алхимия
        u'\U0001F780-\U0001F7FF'  # геометрические фигуры
        u'\U0001F800-\U0001F8FF'  # дополнительные символы
        u'\U0001F900-\U0001F9FF'  # дополнительные символы-2
        u'\U0001FA00-\U0001FA6F'  # шахматы
        u'\U0001FA70-\U0001FAFF'  # дополнительные символы-3
        u'\U00002702-\U000027B0'  # Dingbats
        u'\U000024C2-\U0001F251'  # Enclosed
        ']+', 
        flags=re.UNICODE
    )
    text = emoji_pattern.sub(' ', text)
    
    # Удаление HTML-сущностей и специальных символов
    text = re.sub(r'&[a-z]+;', ' ', text)
    
    # Удаление пунктуации, цифр и не-буквенных символов
    text = re.sub(r'[^a-zа-я\s]', ' ', text)
    
    # Удаление телеграм-упоминаний и бот-команд
    text = re.sub(r'@\w+|/\w+', ' ', text)
    
    # Удаление лишних пробелов и обрезка
    return re.sub(r'\s+', ' ', text).strip()


# Предобработка и индексация документов
processed_data = []
tokenized_corpus = []

for doc_id, text in enumerate(documents):
    # Предобработка текста
    clean_text = preprocess_text(text)
    
    # Обработка с Natasha
    doc = Doc(clean_text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    
    # Лемматизация
    lemmas = []
    for token in doc.tokens:
        if token.pos != 'PUNCT':
            token.lemmatize(morph_vocab)
            lemmas.append(token.lemma)
    
    print("Леммы:", lemmas)
    
    # Для BM25
    tokenized_corpus.append(lemmas.copy())
    
    # Извлечение триплетов
    triplets = []
    for sent in doc.sents:
        subj = [token.lemma for token in sent.tokens if token.rel == 'nsubj']
        obj = [token.lemma for token in sent.tokens if token.rel == 'obj']
        verb = [token.lemma for token in sent.tokens if token.rel == 'root']
        
        if subj and verb and obj:
            triplets.append((
                ' '.join(subj),
                ' '.join(verb),
                ' '.join(obj)
            ))
    
    # Сохранение данных
    processed_data.append({
        'original': text,
        'lemmas': ' '.join(lemmas),
        'triplets': triplets,
        'embedding': sbert_model.encode(text)
    })

    print("Извлеченные триплеты:", triplets)

# Инициализация BM25
bm25 = BM25Okapi(tokenized_corpus)

def hybrid_search(query, top_n=3):
    """Гибридный поиск с использованием BM25, триплетов и SBERT"""
    # Предобработка запроса
    clean_query = preprocess_text(query)
    
    # Обработка запроса с Natasha
    query_doc = Doc(clean_query)
    query_doc.segment(segmenter)
    query_doc.tag_morph(morph_tagger)
    query_doc.parse_syntax(syntax_parser)
    
    # Лемматизация запроса
    query_lemmas = []
    for token in query_doc.tokens:
        if token.pos != 'PUNCT':
            token.lemmatize(morph_vocab)
            query_lemmas.append(token.lemma)
    
    # Поиск по BM25
    bm25_scores = bm25.get_scores(query_lemmas)
    bm25_indices = np.argsort(bm25_scores)[-top_n*2:][::-1]
    
    # Извлечение триплетов запроса
    query_triplets = []
    for sent in query_doc.sents:
        subj = [token.lemma for token in sent.tokens if token.rel == 'nsubj']
        obj = [token.lemma for token in sent.tokens if token.rel == 'obj']
        verb = [token.lemma for token in sent.tokens if token.rel == 'root']
        
        if subj and verb and obj:
            query_triplets.append((
                ' '.join(subj),
                ' '.join(verb),
                ' '.join(obj)
            ))
    
    # Поиск по триплетам в топе BM25
    triplet_matches = []
    for idx in bm25_indices:
        doc = processed_data[idx]
        match_count = sum(1 for t1 in query_triplets for t2 in doc['triplets'] if t1 == t2)
        if match_count > 0:
            triplet_matches.append(idx)
    
    # Семантический поиск
    query_embedding = sbert_model.encode(query)
    similarities = [
        cosine_similarity([query_embedding], [doc['embedding']])[0][0]
        for doc in processed_data
    ]
    
    # Комбинирование результатов
    results = []
    for doc_id in bm25_indices:
        score = 0.3*bm25_scores[doc_id] + 0.7*similarities[doc_id]
        if doc_id in triplet_matches:
            score *= 1.2  # Бонус за совпадение триплетов
        results.append((doc_id, score))
    
    # Сортировка и удаление дубликатов
    seen = set()
    final_results = []
    for doc_id, score in sorted(results, key=lambda x: -x[1]):
        if doc_id not in seen:
            seen.add(doc_id)
            final_results.append((doc_id, score))
            if len(final_results) >= top_n:
                break
    
    return [(processed_data[i]['original'], score) for i, score in final_results]



Леммы: ['твой', 'хороший', 'секс', 'спрятать', 'здесь', 'делиться', 'канал', 'дипломированный', 'сексолог', 'крис', 'взломать', 'код', 'классный', 'секс', 'мастерски', 'раскрепощать', 'знать', 'миллион', 'горячий', 'техника', 'и', 'хороший', 'девайс', 'для', 'взрослый', 'самый', 'полезный', 'пост', 'здесь', 'отрезвлять', 'пост', 'я', 'весь', 'сам', 'прокачать', 'наездница', 'ролевый', 'игра', 'vip', 'кинотеатр', 'техника', 'оральный', 'ласка', 'как', 'заниматься', 'с', 'e', 'ксом', 'неудобный', 'женщина', 'кстати', 'крис', 'провести', 'трехдневный', 'безоплатный', 'онлайн', 'интенсив', 'от', 'бревно', 'до', 'богиня', 'совместно', 'с', 'врач', 'и', 'владелец', 'секс', 'шоп', 'скорый', 'смотри', 'запись', 'пока', 'не', 'удалить', 'здесь', 'жарче', 'чем', 'в', 'ад']
Извлеченные триплеты: [('крис я игра техника женщина крис', 'спрятать взломать знать провести удалить', 'код раскрепощать техника пост пост ласка интенсив запись')]
Леммы: ['кнопка', 'start']
Извлеченные триплеты: []
Леммы: ['

In [14]:
# Пример поиска
query = "звонить"
print("Результаты поиска для запроса:", query)
for i, (text, score) in enumerate(hybrid_search(query)):
    print(f"\nРезультат {i+1} (рейтинг: {score:.2f}):")
    print(text)

Результаты поиска для запроса: звонить

Результат 1 (рейтинг: 0.26):
⭐️  Кнопка: ⭐️START⭐️(https://t.me/major/start?startapp=1972869792)

Результат 2 (рейтинг: 0.12):
ты не мог бы набрать меня после обеда

Результат 3 (рейтинг: 0.10):
он вообще не собирается переезжать в другое государство


In [15]:
# Пример поиска
query = "гадалка"
print("Результаты поиска для запроса:", query)
for i, (text, score) in enumerate(hybrid_search(query)):
    print(f"\nРезультат {i+1} (рейтинг: {score:.2f}):")
    print(text)

Результаты поиска для запроса: гадалка

Результат 1 (рейтинг: 0.08):
⭐️  Кнопка: ⭐️START⭐️(https://t.me/major/start?startapp=1972869792)

Результат 2 (рейтинг: 0.01):
Твой лучший секс спрятан здесь 🔞  Делюсь каналом дипломированного сексолога. Крис взломала код классного секса, мастерски раскрепощает, знает миллион горячих техник и лучшие девайсы для взрослых 😻  Самые полезные посты здесь:   Отрезвляющий пост «Я все сама!»   Прокачай наездницу  Ролевая игра «VIP кинотеатр»   Техника оральных ласк 💣   Как занимается сeксом неудобная женщина   Кстати, Крис провела трехдневный безоплатный онлайн интенсив-«От бревна до Богини». Совместно с врачом и владельцем секс-шопа.   Скорее смотри записи, пока не удалила 🔞  https://t.me/sekretskris/1048   Здесь жарче, чем в аду 😈

Результат 3 (рейтинг: 0.01):
ты не мог бы набрать меня после обеда


In [16]:
# Пример поиска
query = "секс"
print("Результаты поиска для запроса:", query)
for i, (text, score) in enumerate(hybrid_search(query)):
    print(f"\nРезультат {i+1} (рейтинг: {score:.2f}):")
    print(text)

Результаты поиска для запроса: секс

Результат 1 (рейтинг: 0.78):
Твой лучший секс спрятан здесь 🔞  Делюсь каналом дипломированного сексолога. Крис взломала код классного секса, мастерски раскрепощает, знает миллион горячих техник и лучшие девайсы для взрослых 😻  Самые полезные посты здесь:   Отрезвляющий пост «Я все сама!»   Прокачай наездницу  Ролевая игра «VIP кинотеатр»   Техника оральных ласк 💣   Как занимается сeксом неудобная женщина   Кстати, Крис провела трехдневный безоплатный онлайн интенсив-«От бревна до Богини». Совместно с врачом и владельцем секс-шопа.   Скорее смотри записи, пока не удалила 🔞  https://t.me/sekretskris/1048   Здесь жарче, чем в аду 😈

Результат 2 (рейтинг: 0.15):
Таро-прогноз на 23 Июля – Туз Мечей.   🔮 Совет: Меч – это достаточное количество сил для энергичных начинаний, огромная решимость действовать, начало успешной борьбы, готовность к росту, отрыву от изначального существования. Это ситуация, когда сам человек для себя многое прояснил и знает теперь

[Установка данных NLTK](https://www.nltk.org/data.html)

In [ ]:
import nltk
nltk.download()

In [ ]:
nltk.download('ru-wordnet') # Загрузка русского WordNet

In [ ]:
!pip install ruwordnet
!ruwordnet download

In [18]:
from ruwordnet import RuWordNet
wn = RuWordNet()

In [19]:
for sense in wn.get_senses('замок'):
    print(sense.synset)
# Synset(id="126228-N", title="СРЕДНЕВЕКОВЫЙ ЗАМОК")
# Synset(id="114707-N", title="ЗАМОК ДЛЯ ЗАПИРАНИЯ")

Synset(id="126228-N", title="СРЕДНЕВЕКОВЫЙ ЗАМОК")
Synset(id="114707-N", title="ЗАМОК ДЛЯ ЗАПИРАНИЯ")


In [20]:
for sense in wn.get_senses('собака'):
    print(sense.synset, [synonym.name for synonym in sense.synset.senses])

Synset(id="4454-N", title="СОБАКА") ['СОБАКА', 'ПЕС', 'СОБАЧКА', 'СОБАЧОНКА', 'ПСИНА', 'ЧЕТВЕРОНОГИЙ ДРУГ', 'ПЕСИК']


In [24]:
# Пример документов для поиска
documents = [
    "Твой лучший секс спрятан здесь 🔞  Делюсь каналом дипломированного сексолога. Крис взломала код классного секса, мастерски раскрепощает, знает миллион горячих техник и лучшие девайсы для взрослых 😻  Самые полезные посты здесь:   Отрезвляющий пост «Я все сама!»   Прокачай наездницу  Ролевая игра «VIP кинотеатр»   Техника оральных ласк 💣   Как занимается сeксом неудобная женщина   Кстати, Крис провела трехдневный безоплатный онлайн интенсив-«От бревна до Богини». Совместно с врачом и владельцем секс-шопа.   Скорее смотри записи, пока не удалила 🔞  https://t.me/sekretskris/1048   Здесь жарче, чем в аду 😈",
    "⭐️  Кнопка: ⭐️START⭐️(https://t.me/major/start?startapp=1972869792)",
    "Таро-прогноз на 23 Июля – Туз Мечей.   🔮 Совет: Меч – это достаточное количество сил для энергичных начинаний, огромная решимость действовать, начало успешной борьбы, готовность к росту, отрыву от изначального существования. Это ситуация, когда сам человек для себя многое прояснил и знает теперь, что хочет делать дальше. Это показатель силы – физической силы, силы воли, силы, приобретенной положением и умом или в силу обстоятельств. Триумф личной силы, власти над обстоятельствами. Триумф может относиться к любой стороне жизни: к работе, любви, денежным делам, духу, любым увлекающим занятиям. Этот Туз не столько начало, но и показатель завоеванного, торжества провозглашенных взглядов и принятых решений.   💰 Деньги: Процветание. Принятие однозначных и окончательных решений в этих вопросах. Возможность улучшить качество жизни, приняв вызов.   🩷 Любовь: В области отношений Туз Мечей символизирует импульсивную первобытную силу, интенсивные «завоевательные» эмоции, крайние чувства, связанные с ситуацией или человеком. Эмоции, которым покровительствует Туз Мечей, способны воспламенить таким огнем, который сожжет все препятствия на пути к цели, попутно причинив немало вреда. Мечи вообще масть холодная, и когда на горизонте в кои-то веки появляется единственная эмоция, то она заполоняет собой всё со свойственной этой масти тотальностью.  🎁🔥 РАСПРОДАЖА КУРСА «ОРАКУЛ ЛЕНОРМАН. БАЗОВЫЙ КУРС» В РАССРОЧКУ 👉🏻 http://alexeygrishin.com/lenorman_base.",
    "он вообще не собирается переезжать в другое государство",
    "ты не мог бы набрать меня после обеда"
]

In [35]:
from natasha import Doc, Segmenter, NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, MorphVocab
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
from sklearn.metrics.pairwise import cosine_similarity
from ruwordnet import RuWordNet
import re
import numpy as np


# Инициализация компонентов
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
morph_vocab = MorphVocab()
# Инициализация RuWordNet
wn = RuWordNet()  
# sbert_model = SentenceTransformer('ai-forever/sbert_large_mt_nlu_ru')
sbert_model = model # Используйте предварительно загруженную модель


# def preprocess_text(text):
#     """Функция предобработки текста"""
#     # Приведение к нижнему регистру и замена ё
#     text = text.lower().replace("ё", "е")
#     # Удаление пунктуации и цифр
#     text = re.sub(r'[^\w\s]|[\d]', ' ', text)
#     # Удаление лишних пробелов
#     return re.sub(r'\s+', ' ', text).strip()

def preprocess_text(text):
    """Функция предобработки текста с расширенной очисткой"""
    # Приведение к нижнему регистру и замена ё
    text = text.lower().replace("ё", "е")
    
    # Удаление URL-ссылок
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
    
    # Удаление эмодзи и специальных символов
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # эмоции
        u'\U0001F300-\U0001F5FF'  # символы
        u'\U0001F680-\U0001F6FF'  # транспорт
        u'\U0001F700-\U0001F77F'  # алхимия
        u'\U0001F780-\U0001F7FF'  # геометрические фигуры
        u'\U0001F800-\U0001F8FF'  # дополнительные символы
        u'\U0001F900-\U0001F9FF'  # дополнительные символы-2
        u'\U0001FA00-\U0001FA6F'  # шахматы
        u'\U0001FA70-\U0001FAFF'  # дополнительные символы-3
        u'\U00002702-\U000027B0'  # Dingbats
        u'\U000024C2-\U0001F251'  # Enclosed
        ']+', 
        flags=re.UNICODE
    )
    text = emoji_pattern.sub(' ', text)
    
    # Удаление HTML-сущностей и специальных символов
    text = re.sub(r'&[a-z]+;', ' ', text)
    
    # Удаление пунктуации, цифр и не-буквенных символов
    text = re.sub(r'[^a-zа-я\s]', ' ', text)
    
    # Удаление телеграм-упоминаний и бот-команд
    text = re.sub(r'@\w+|/\w+', ' ', text)
    
    # Удаление лишних пробелов и обрезка
    return re.sub(r'\s+', ' ', text).strip()


# Предобработка и индексация документов
processed_data = []
tokenized_corpus = []

for doc_id, text in enumerate(documents):
    # Предобработка текста
    clean_text = preprocess_text(text)
    
    # Обработка с Natasha
    doc = Doc(clean_text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    
    # Лемматизация
    lemmas = []
    for token in doc.tokens:
        if token.pos != 'PUNCT':
            token.lemmatize(morph_vocab)
            lemmas.append(token.lemma)
    
    print("Леммы:", lemmas)
    
    # Для BM25
    tokenized_corpus.append(lemmas.copy())
    
    # Извлечение триплетов
    triplets = []
    for sent in doc.sents:
        subj = [token.lemma for token in sent.tokens if token.rel == 'nsubj']
        obj = [token.lemma for token in sent.tokens if token.rel == 'obj']
        verb = [token.lemma for token in sent.tokens if token.rel == 'root']
        
        if subj and verb and obj:
            triplets.append((
                ' '.join(subj),
                ' '.join(verb),
                ' '.join(obj)
            ))
    
    # Сохранение данных
    processed_data.append({
        'original': text,
        'lemmas': ' '.join(lemmas),
        'triplets': triplets,
        'embedding': sbert_model.encode(text)
    })

    print("Извлеченные триплеты:", triplets)

# Инициализация BM25
bm25 = BM25Okapi(tokenized_corpus)

def expand_with_ruwordnet(lemmas):
    """Расширение лемм с помощью RuWordNet"""
    expanded = []
    for lemma in lemmas:
        expanded.append(lemma)
        # Поиск синсетов для леммы
        synsets = wn.get_synsets(lemma)
        for synset in synsets:
            # Добавляем все синонимы из синсета
            expanded.extend([sense.name.lower() for sense in synset.senses])
    return list(set(expanded))  # Удаляем дубликаты

def hybrid_search(query, top_n=3):
    """Гибридный поиск с расширением синонимов"""
    # Предобработка запроса
    clean_query = preprocess_text(query)
    
    # Обработка запроса с Natasha
    query_doc = Doc(clean_query)
    query_doc.segment(segmenter)
    query_doc.tag_morph(morph_tagger)
    query_doc.parse_syntax(syntax_parser)
    
    # Лемматизация запроса
    query_lemmas = []
    for token in query_doc.tokens:
        if token.pos != 'PUNCT':
            token.lemmatize(morph_vocab)
            query_lemmas.append(token.lemma.lower())
    
    # Расширение синонимами RuWordNet
    expanded_lemmas = expand_with_ruwordnet(query_lemmas)
    print(f"Расширенные леммы: {expanded_lemmas}")
    
    # Поиск по BM25 с расширенными леммами
    bm25_scores = bm25.get_scores(expanded_lemmas)
    bm25_indices = np.argsort(bm25_scores)[-top_n*2:][::-1]
    
    # Извлечение триплетов запроса
    query_triplets = []
    for sent in query_doc.sents:
        subj = [token.lemma for token in sent.tokens if token.rel == 'nsubj']
        obj = [token.lemma for token in sent.tokens if token.rel == 'obj']
        verb = [token.lemma for token in sent.tokens if token.rel == 'root']
        
        if subj and verb and obj:
            query_triplets.append((
                ' '.join(subj),
                ' '.join(verb),
                ' '.join(obj)
            ))
    
    # Поиск по триплетам в топе BM25
    triplet_matches = []
    for idx in bm25_indices:
        doc = processed_data[idx]
        match_count = sum(1 for t1 in query_triplets for t2 in doc['triplets'] if t1 == t2)
        if match_count > 0:
            triplet_matches.append(idx)
    
    # Семантический поиск
    query_embedding = sbert_model.encode(query)
    similarities = [
        cosine_similarity([query_embedding], [doc['embedding']])[0][0]
        for doc in processed_data
    ]
    
    # Комбинирование результатов
    # results = []
    # for doc_id in bm25_indices:
    #     score = 0.3*bm25_scores[doc_id] + 0.7*similarities[doc_id]
    #     if doc_id in triplet_matches:
    #         score *= 1.2  # Бонус за совпадение триплетов
    #     results.append((doc_id, score))
    
    # Комбинирование результатов
    results = []
    for doc_id in bm25_indices:
        doc = processed_data[doc_id]
        
        # Бонусы
        length_boost = 1.5 if len(doc['lemmas'].split()) < 10 else 1.0
        exact_match = 1.2 if any(lem in expanded_lemmas for lem in doc['lemmas'].split()) else 1.0
        
        score = (
            0.3 * bm25_scores[doc_id] +
            0.7 * cosine_similarity([query_embedding], [doc['embedding']])[0][0]
        ) * length_boost * exact_match
        
        results.append((doc_id, score))

    # Сортировка и удаление дубликатов
    seen = set()
    final_results = []
    for doc_id, score in sorted(results, key=lambda x: -x[1]):
        if doc_id not in seen:
            seen.add(doc_id)
            final_results.append((doc_id, score))
            if len(final_results) >= top_n:
                break
    
    return [(processed_data[i]['original'], score) for i, score in final_results]



Леммы: ['доченька', 'твой', 'совсем', 'больший', 'стать']
Извлеченные триплеты: []
Леммы: ['весь', 'дорога', 'забить', 'дерево', 'и', 'цвет']
Извлеченные триплеты: []
Леммы: ['в', 'следующий', 'воскресение', 'я', 'собираться', 'в', 'питер']
Извлеченные триплеты: []
Леммы: ['у', 'я', 'сломаться', 'стиралка', 'прикинуть']
Извлеченные триплеты: []
Леммы: ['садиться', 'в', 'машина', 'и', 'поехать', 'уже']
Извлеченные триплеты: []
Леммы: ['сколько', 'стоить', 'ремонт', 'стиральный', 'машина']
Извлеченные триплеты: []
Леммы: ['ты', 'когда', 'собираться', 'звонить', 'препод']
Извлеченные триплеты: []
Леммы: ['взять', 'пистолет', 'в', 'тумбочка', 'понять', 'я']
Извлеченные триплеты: []
Леммы: ['ты', 'не', 'мочь', 'бы', 'набрать', 'я', 'после', 'обед']
Извлеченные триплеты: [('ты', 'мочь', 'я')]
Леммы: ['ты', 'взять', 'корзина', 'прежде', 'чем', 'набрать', 'продукт']
Извлеченные триплеты: [('ты', 'взять', 'корзина продукт')]
Леммы: ['он', 'сегодня', 'утро', 'отвезти', 'в', 'близкий', 'госпиталь

In [30]:
# Пример поиска
query = "звонить"
print("Результаты поиска для запроса:", query)
for i, (text, score) in enumerate(hybrid_search(query)):
    print(f"\nРезультат {i+1} (рейтинг: {score:.2f}):")
    print(text)

Результаты поиска для запроса: звонить
Расширенные леммы: ['позвонить по телефону', 'говорить по телефону', 'звякнуть', 'отзванивать', 'созваниваться', 'издавать звон', 'созвониться', 'зазвонить', 'беседовать по телефону', 'звякнуть по телефону', 'издать звон', 'разговаривать по телефону', 'пользоваться телефоном', 'отзвониться', 'звонить', 'перезваниваться', 'звонить по телефону', 'позвонить', 'названивать', 'отзвонить']

Результат 1 (рейтинг: 0.39):
⭐️  Кнопка: ⭐️START⭐️(https://t.me/major/start?startapp=1972869792)

Результат 2 (рейтинг: 0.18):
ты не мог бы набрать меня после обеда

Результат 3 (рейтинг: 0.15):
он вообще не собирается переезжать в другое государство


In [31]:
# Пример поиска
query = "гадалка"
print("Результаты поиска для запроса:", query)
for i, (text, score) in enumerate(hybrid_search(query)):
    print(f"\nРезультат {i+1} (рейтинг: {score:.2f}):")
    print(text)

Результаты поиска для запроса: гадалка
Расширенные леммы: ['гадалка', 'ворожея']

Результат 1 (рейтинг: 0.12):
⭐️  Кнопка: ⭐️START⭐️(https://t.me/major/start?startapp=1972869792)

Результат 2 (рейтинг: 0.01):
ты не мог бы набрать меня после обеда

Результат 3 (рейтинг: 0.01):
Твой лучший секс спрятан здесь 🔞  Делюсь каналом дипломированного сексолога. Крис взломала код классного секса, мастерски раскрепощает, знает миллион горячих техник и лучшие девайсы для взрослых 😻  Самые полезные посты здесь:   Отрезвляющий пост «Я все сама!»   Прокачай наездницу  Ролевая игра «VIP кинотеатр»   Техника оральных ласк 💣   Как занимается сeксом неудобная женщина   Кстати, Крис провела трехдневный безоплатный онлайн интенсив-«От бревна до Богини». Совместно с врачом и владельцем секс-шопа.   Скорее смотри записи, пока не удалила 🔞  https://t.me/sekretskris/1048   Здесь жарче, чем в аду 😈


In [34]:
# Пример документов для поиска
documents = [
    "доченька твоя совсем большая стала",
    "вся дорога забита деревьями и цветами",
    "в следующее воскресенье я собираюсь в питер",
    "у меня сломалась стиралка прикинь",
    "садись в машину и поехали уже",
    "сколько стоит ремонт стиральной машины",
    "ты когда собираешься звонить преподу",
    "возьмешь пистолет в тумбочке понял меня",
    "ты не мог бы набрать меня после обеда",
    "ты возьми корзину прежде чем набрать продукты",
    "его сегодня утром отвезли в ближайший госпиталь",
    "в этом году смартфоны подорожают на 30 процентов",
    "я еще долгу не смогу вернуться в рф",
    "мужчина средних лет с серым рюкзаком и шапкой",
    "в какой университет собирается поступать твой сын",
    "он вообще не собирается переезжать в другое государство",
    "сегодня ночью наше судно отплывает по расписанию",
    "его оригинальный портрет выставлен на продажу"
]


In [36]:
# Пример поиска
query = "звонить"
print("Результаты поиска для запроса:", query)
for i, (text, score) in enumerate(hybrid_search(query)):
    print(f"\nРезультат {i+1} (рейтинг: {score:.2f}):")
    print(text)

Результаты поиска для запроса: звонить
Расширенные леммы: ['позвонить по телефону', 'говорить по телефону', 'звякнуть', 'отзванивать', 'созваниваться', 'издавать звон', 'созвониться', 'зазвонить', 'беседовать по телефону', 'звякнуть по телефону', 'издать звон', 'разговаривать по телефону', 'пользоваться телефоном', 'отзвониться', 'звонить', 'перезваниваться', 'звонить по телефону', 'позвонить', 'названивать', 'отзвонить']

Результат 1 (рейтинг: 1.89):
ты когда собираешься звонить преподу

Результат 2 (рейтинг: 0.15):
в какой университет собирается поступать твой сын

Результат 3 (рейтинг: 0.15):
он вообще не собирается переезжать в другое государство


In [37]:
# Пример поиска
query = "подлинная картина"
print("Результаты поиска для запроса:", query)
for i, (text, score) in enumerate(hybrid_search(query)):
    print(f"\nРезультат {i+1} (рейтинг: {score:.2f}):")
    print(text)

Результаты поиска для запроса: подлинная картина
Расширенные леммы: ['реальная картина', 'картина спектакля', 'киноальманах', 'кинематографическая продукция', 'произведение живописи', 'живопись', 'кинокартина', 'подлинный', 'кинопродукция', 'кинопроизведение', 'зрелище', 'картина художника', 'киноработа', 'киношка', 'кинематографическое произведение', 'полотно художника', 'словесная картина', 'киноэпопея', 'полотно', 'кинолента', 'картина', 'картина положения', 'неподдельный', 'живописное произведение', 'живописная работа', 'киносериал', 'картина происходящего', 'настоящий', 'живописное полотно', 'кинофильм', 'полная картина', 'картина состояния', 'общая картина', 'продукция киностудий', 'действительный', 'целостная картина']

Результат 1 (рейтинг: 0.57):
его оригинальный портрет выставлен на продажу

Результат 2 (рейтинг: 0.14):
сегодня ночью наше судно отплывает по расписанию

Результат 3 (рейтинг: 0.08):
я еще долгу не смогу вернуться в рф


In [38]:
# Пример поиска
query = "стиральная машина"
print("Результаты поиска для запроса:", query)
for i, (text, score) in enumerate(hybrid_search(query)):
    print(f"\nРезультат {i+1} (рейтинг: {score:.2f}):")
    print(text)

Результаты поиска для запроса: стиральная машина
Расширенные леммы: ['технический инструмент', 'технологическое устройство', 'техническое приспособление', 'машинная продукция', 'машинка', 'стиральный', 'машинно-техническая продукция', 'технический аппарат', 'авто', 'средство автотранспорта', 'автомашина', 'машина', 'механизм', 'устройство', 'техника', 'аппаратура', 'техническое средство', 'автомобиль', 'техническое устройство', 'техническое оборудование', 'техническая система', 'автотранспорт', 'автотранспортное средство', 'автосредство', 'аппарат', 'техническая аппаратура', 'аппаратное средство', 'технологическая система', 'техсредство', 'система']

Результат 1 (рейтинг: 3.36):
сколько стоит ремонт стиральной машины

Результат 2 (рейтинг: 1.23):
садись в машину и поехали уже

Результат 3 (рейтинг: 0.21):
сегодня ночью наше судно отплывает по расписанию


In [39]:
# Пример поиска
query = "оружие"
print("Результаты поиска для запроса:", query)
for i, (text, score) in enumerate(hybrid_search(query)):
    print(f"\nРезультат {i+1} (рейтинг: {score:.2f}):")
    print(text)

Результаты поиска для запроса: оружие
Расширенные леммы: ['оружие для достижения цели', 'орудие для достижения', 'орудие для достижения цели', 'оружие', 'оружие для достижения']

Результат 1 (рейтинг: 0.20):
мужчина средних лет с серым рюкзаком и шапкой

Результат 2 (рейтинг: 0.17):
в какой университет собирается поступать твой сын

Результат 3 (рейтинг: 0.14):
сегодня ночью наше судно отплывает по расписанию
